In [2]:
import numpy as np
import pandas as pd
import plotly.graph_objects as go
import plotly.express as px
from plotly.subplots import make_subplots
import matplotlib.pyplot as plt

# Custom libraries
from Components.TrainModel import DataModule, TEMPUS, torchscript_predict
from Components.TickerData import TickerData, upload_data_sql, fetch_sql_data
from Components.BackTesting import BackTesting
from Components.MarketRegimes import MarketRegimes

# Torch ML libraries
import torch
import torch.nn as nn
from torch.optim import AdamW

device = "cuda" if torch.cuda.is_available() else "cpu"
if device == "cuda":
    torch.backends.cuda.matmul.allow_tf32 = True
    torch.backends.cudnn.allow_tf32 = True

In [2]:
#TODO: Include alpha in the backtesting results (based on index comparison) from quantstats package
#TODO: Streamlit Page for future prediction

# -- Model Features/Data --
#TODO: Use following news sentiment features: ["positive_count","neutral_count","negative_count","total_count","pos_sent_ratio","neg_sent_ratio","net_sentiment"]
#TODO: Parse financials data from Pologon.IO
#TODO: Include index volitility in the training data

# -- Model Training --

In [3]:
# Set the Wikipedia page title and section header
sample_size = 50

nasdaq_tickers = pd.read_html("https://en.wikipedia.org/wiki/Nasdaq-100")[4]
nasdaq_tickers = nasdaq_tickers.iloc[:, [1]].to_numpy().flatten()
nasdaq_tickers = np.random.choice(nasdaq_tickers, size=sample_size, replace=False)
rusell_tickers = pd.read_html("https://en.wikipedia.org/wiki/Russell_1000_Index")[3]
rusell_tickers = rusell_tickers.iloc[:, [1]].to_numpy().flatten()
rusell_tickers = np.random.choice(rusell_tickers, size=sample_size, replace=False)
SnP500_tickers = pd.read_html("https://en.wikipedia.org/wiki/List_of_S%26P_500_companies")[0]
SnP500_tickers = SnP500_tickers.iloc[:, [0]].to_numpy().flatten()
SnP500_tickers = np.random.choice(SnP500_tickers, size=sample_size, replace=False)
SnP600_tickers = pd.read_html("https://en.wikipedia.org/wiki/List_of_S%26P_600_companies")[0]
SnP600_tickers = SnP600_tickers.iloc[:, [0]].to_numpy().flatten()
SnP600_tickers = np.random.choice(SnP600_tickers, size=sample_size, replace=False)

tickers = np.concatenate((nasdaq_tickers,rusell_tickers,SnP500_tickers,SnP600_tickers))
tickers = np.unique(tickers)

In [4]:
#tickers = ['IONQ']
indicators = ['ema_20', 'ema_50', 'ema_200', 'stoch_rsi14','stoch_rsi28','nasdaq_rsi14','macd', 'b_percent', 'keltner_lower', 'keltner_upper','State','bearish','bullish','hold','mixed','negative','neutral','positive','z_score','atr','price_momentum','volume_momentum','Close']
#indicators = ['bearish','bullish','hold','mixed','negative','neutral','positive']
training_data, raw_stock_data = TickerData(tickers,years=5,prediction_window=5,indicator_list=indicators).process_all()
training_data

MaxRetryError: HTTPSConnectionPool(host='api.polygon.io', port=443): Max retries exceeded with url: /v2/aggs/ticker/I:NDX/range/1/day/2020-04-27/2025-04-26?limit=50000 (Caused by ResponseError('too many 429 error responses'))

In [ ]:
#Best config: {'lr': 4.390449033248878e-05, 'hidden_size': 256, 'num_layers': 1, 'dropout': 0.3477694988633191, 'weight_decay': 0.0001801390872725824, 'batch_size': 16, 'window_size': 10, 'grad_clip_norm': 0.8393802881451728}

config = {
    "lr": 4.390449033248878e-05,
    "weight_decay": 0.0001801390872725824,
    "hidden_size": 256, # old was 256
    "num_layers": 1, # old was 1
    "dropout": 0.3477694988633191,
    "batch_size": 16, # old was 16
    "window_size": 5,
    "clip_size": 0.8393802881451728,
    "attention_heads": 4, #Deepseek R1 uses 128
    "epochs": 20,
    "device": "cuda" if torch.cuda.is_available() else "mps"
}

data_module = DataModule(training_data, window_size=config["window_size"], batch_size=config["batch_size"])
config["input_size"] = data_module.num_features

# Instantiate the model
model = TEMPUS(config,scaler=data_module.scaler).to(config["device"])
#model = torch.compile(model, backend="inductor",mode="default")
model
# Train Model
history = model.train_model(data_module.train_loader, data_module.val_loader, data_module.test_loader, config["epochs"])

In [ ]:
training_fig = model.plot_training_history()
training_fig.show()

In [ ]:
# Export the trained TEMPUS model
script_path = model.export_model_to_torchscript(
    save_path="Models/Echo_v1.0.pt",
    data_loader=data_module.test_loader,
    device="cpu"
)

In [ ]:
import random
# Randomly sample 50 tickers from the SnP600_tickers list
sampled_tickers = random.sample(list(nasdaq_tickers), 10)
initial_capital = 1000.0

preds_dfs = []
returns = []
for idx, ticker in enumerate(sampled_tickers, start=1):
    out_of_sample_data, raw_stock_data = TickerData(ticker, years=4, prediction_window=5).process_all()

    # Check if raw_stock_data is NoneType, if so, skip this iteration
    if out_of_sample_data is not None:
        # Load the model and make predictions
        preds_df = torchscript_predict(
            model_path="Models/Tempus_v2.1.pt",
            input_df=out_of_sample_data,
            device="cpu",
            window_size=50,
            target_col="shifted_prices"
        )
        preds_df = pd.merge(preds_df, raw_stock_data[['Open', 'High', 'Low', 'Volume','Close']], left_index=True, right_index=True, how='left')
        preds_dfs.append(preds_df)

        backtester = BackTesting(preds_df, ticker, initial_capital, pct_change_entry=0.05, pct_change_exit=0.03)
        backtester.run_simulation()
        bt_results = pd.DataFrame(backtester.pf.returns())
        bt_results['cumulative_return'] = np.array(((1 + bt_results[0]).cumprod() - 1)*100)
        bt_results['ticker'] = ticker
        returns.append(bt_results)

preds_dfs = pd.concat(preds_dfs, ignore_index=False)
returns = pd.concat(returns, ignore_index=False)

In [ ]:
# Calculate cumulative returns for each ticker and visualize them using Plotly
# Group data by 'ticker' and calculate cumulative returns

# Create an interactive plot using Plotly
fig = px.line(
    returns.reset_index(),
    x='index',
    y='cumulative_return',
    color='ticker',
    title='Cumulative Returns by Ticker',
    labels={'index': 'Date', 'cumulative_return': 'Cumulative Return'}
)

fig.update_layout(
    xaxis_title='Date',
    yaxis_title='Cumulative Return (%)',
    showlegend=False,
    height=600,
    template='ggplot2',
    xaxis=dict(
        rangeselector=dict(
            buttons=list([
                dict(count=1, label="1m", step="month", stepmode="backward"),
                dict(count=6, label="6m", step="month", stepmode="backward"),
                dict(count=1, label="YTD", step="year", stepmode="todate"),
                dict(count=1, label="1y", step="year", stepmode="backward"),
                dict(step="all")
            ])
        ),
        rangeslider=dict(visible=False),
        type="date"
    )
)
fig.show()

last_returns = returns.groupby('ticker')['cumulative_return'].last()

# Count positive and negative returns
positive_count = sum(last_returns > 0)
negative_count = sum(last_returns <= 0)
total_count = len(last_returns)

# Convert to DataFrame for visualization
last_returns_df = pd.DataFrame(last_returns).reset_index()
last_returns_df.columns = ['Ticker', 'Final Return']
last_returns_df.sort_values('Final Return', ascending=False, inplace=True)

# Create a simple pie chart showing the proportion
fig_pie = px.pie(
    values=[positive_count, negative_count],
    names=['Positive', 'Negative'],
    title='Proportion of Tickers with Positive vs Negative Returns',
    color_discrete_sequence=['green', 'red'],
    template='ggplot2',
)

fig_pie.update_traces(textinfo='percent+label').update_layout(showlegend=False)
fig_pie.show()

# Calculate the proportion of tickers with positive returns
if total_count > 0:
    positive_proportion = positive_count / total_count
    print(f"Proportion of tickers with positive cumulative returns: {positive_proportion:.2%}")
    print(f"Positive tickers: {positive_count} out of {total_count}")
    print(f"Negative tickers: {negative_count} out of {total_count}")
else:
    print("No ticker data available for analysis")

In [ ]:
# Randomly select a ticker from the `preds_dfs` object
selected_ticker = random.choice(preds_dfs['Ticker'].unique())

# Filter the `preds_dfs` DataFrame for the selected ticker
preds_df = preds_dfs[preds_dfs['Ticker'] == selected_ticker]

# Update the plot to reflect the filtered data
fig = go.Figure()
fig.add_trace(go.Scatter(y=preds_df['Predicted'], x=preds_df.index, mode='lines', name='Predicted', line=dict(color="Grey")))
fig.add_trace(go.Scatter(y=preds_df['Close'], x=preds_df.index, mode='lines', name='Close (Unshifted)', line=dict(color="Blue")))
fig.add_trace(go.Scatter(y=preds_df['Actual'], x=preds_df.index, mode='lines', name='Close (Shifted)'))
fig.update_layout(
    title=f'Prediction for {selected_ticker}',
    xaxis_title='Date',
    yaxis_title='Price (USD)',
    height=600,
    legend=dict(orientation="h", yanchor="bottom", y=1.02),
    template='ggplot2'
)
fig.show()

In [ ]:
from Components.BackTesting import BackTesting
import pandas as pd
ticker = 'PLTR'
out_of_sample_data, raw_stock_data = TickerData(ticker, years=1, prediction_window=5,prediction_mode=True).process_all()

preds_df = torchscript_predict(
    model_path="Models/Tempus_v2.2.pt",
    input_df=out_of_sample_data,
    device="cpu",
    window_size=50,
    prediction_mode=True
)
preds_df = pd.merge(preds_df, raw_stock_data[['Open', 'High', 'Low', 'Volume','Close']], left_index=True, right_index=True, how='left')
preds_df['shifted_prices'] = preds_df['Close'].shift(-abs(5))

In [ ]:
fig = go.Figure()
fig.add_trace(go.Scatter(y=preds_df['Predicted'], x=preds_df.index, mode='lines', name='Predicted', line=dict(color="Grey")))
fig.add_trace(go.Scatter(y=preds_df['shifted_prices'], x=preds_df.index, mode='lines', name='Close (Shifted)', line=dict(color="Blue")))
fig.add_trace(go.Scatter(y=preds_df['Close'], x=preds_df.index, mode='lines', name='Close (Unshifted)', line=dict(color="Orange")))
fig.update_layout(template='ggplot2')
fig.show()

In [ ]:
import quantstats as qs

backtester = BackTesting(preds_df, ticker, initial_capital, pct_change_entry=0.05,pct_change_exit=0.02)
backtester.run_simulation()
returns = backtester.pf.returns()
returns.index = returns.index.tz_localize(None)

#html = qs.reports.full(returns, "NDAQ")
qs.reports.basic(returns, "PLTR",rf=0.0025, display=False)


In [ ]:
df_sentiment = training_data
df_sentiment['total_count'] = df_sentiment['positive'] + df_sentiment['negative'] + df_sentiment['neutral']
df_sentiment['pos_sent_ratio'] = df_sentiment['positive'] / df_sentiment['total_count']
df_sentiment['neg_sent_ratio'] = df_sentiment['negative'] / df_sentiment['total_count']
df_sentiment['neu_sent_ratio'] = df_sentiment['neutral'] / df_sentiment['total_count']
df_sentiment['net_sentiment'] = (df_sentiment['positive'] - df_sentiment['negative']) / df_sentiment['total_count']
df_sentiment['roll3_net'] = df_sentiment['net_sentiment'].rolling(3).mean()
df_sentiment.fillna(0, inplace=True)
df_sentiment

In [ ]:
import numpy as np
import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader
from sklearn.model_selection import train_test_split
import shap
import matplotlib.pyplot as plt
import warnings
warnings.filterwarnings('ignore')

# ————————————
# 1) Prepare your data
# ————————————
data_module = DataModule(training_data, window_size=10, batch_size=32,target_col='shifted_prices')
# ————————————
# 2) Define the LSTM model
# ————————————
class LSTMRegressor(nn.Module):
    def __init__(self, input_dim, hidden_dim=64, num_layers=1, dropout=0.2):
        super().__init__()
        self.lstm = nn.LSTM(
            input_size=input_dim,
            hidden_size=hidden_dim,
            num_layers=num_layers,
            batch_first=True,
            dropout=dropout
        )
        self.out = nn.Linear(hidden_dim, 1)

    def forward(self, x):
        # x: (batch, seq_len, features)
        _, (hn, _) = self.lstm(x)
        # take the last layer’s hidden state
        h_last = hn[-1]               # shape (batch, hidden_dim)
        return self.out(h_last).squeeze(-1)

device = torch.device("cuda" if torch.cuda.is_available() else "mps" ) #mps
model = LSTMRegressor(input_dim=data_module.num_features).to(device)
optimizer = torch.optim.Adam(model.parameters(), lr=1e-3)
criterion = nn.MSELoss()

# ————————————
# 3) Training loop
# ————————————
EPOCHS = 20
for epoch in range(1, EPOCHS+1):
    model.train()
    total_loss = 0
    for xb, yb in data_module.train_loader:
        xb, yb = xb.to(device), yb.to(device)
        optimizer.zero_grad()
        pred = model(xb)
        if pred.dim() > 1:
            pred = pred[:, -1, 0] if pred.size(1) > 1 and pred.size(2) > 0 else pred.squeeze()
        loss = criterion(pred, yb)
        loss.backward()
        optimizer.step()
        total_loss += loss.item() * xb.size(0)

    avg_loss = total_loss / len(data_module.train_loader.dataset)
    print(f"Epoch {epoch:2d} — Train MSE: {avg_loss}")

In [ ]:
# ————————————
# 4) SHAP feature‐importance
# ————————————
import numpy as np
import shap
import torch
import matplotlib.pyplot as plt

# 1) Pull one batch from each loader
#    Assumes each batch is a tuple (x, y) where
#      x.shape == [batch_size, seq_len, n_features]
train_batch, _ = next(iter(data_module.train_loader))
test_batch,  _ = next(iter(data_module.test_loader))

# Move to CPU / numpy for KernelExplainer
# and infer seq_len & n_features
train_np = train_batch.cpu().numpy()
test_np  = test_batch.cpu().numpy()
batch_size, seq_len, n_features = train_np.shape

# 2) Build your background set (<=100 samples), flattened
bg       = train_np[:100]                             # (B_bg, seq_len, n_feat)
bg_flat  = bg.reshape(bg.shape[0], -1)                # (B_bg, seq_len*n_feat)

# flatten all of test for explainer
test_flat = test_np.reshape(test_np.shape[0], -1)     # (batch_size, seq_len*n_feat)

# 3) Define a “flatten→3D→model→1D” wrapper
def predict_flat(x_flat):
    # x_flat: array shape (B, seq_len*n_feat) or (seq_len*n_feat,)
    arr = np.array(x_flat)
    arr = arr.reshape(-1, seq_len, n_features)        # back to (B, seq_len, n_feat)
    t   = torch.from_numpy(arr).float().to(device)
    model.eval()
    with torch.no_grad():
        out = model(t).cpu().numpy().reshape(-1)      # (B,) scalar outputs
    return out

# 4) KernelExplainer on the flattened background
explainer      = shap.KernelExplainer(predict_flat, bg_flat)

# pick up to 50 test windows, but don’t assume you actually have 50
n_explain = min(50, test_np.shape[0])
shap_vals_flat = explainer.shap_values(test_flat[:n_explain])

# if multi-output, grab the first list-element
if isinstance(shap_vals_flat, list):
    shap_vals_flat = shap_vals_flat[0]

# now reshape using the correct n_explain
shap_vals = np.array(shap_vals_flat).reshape(n_explain, seq_len, n_features)

# 6) collapse the time axis and plot
mean_abs_time = np.mean(np.abs(shap_vals), axis=0)  # (seq_len, n_feat)
feat_imp      = mean_abs_time.mean(axis=0)         # (n_feat,)

feature_names = list(training_data.drop(columns=["shifted_prices"]).columns)

fig = go.Figure()
fig.add_trace(go.Bar(y=feat_imp, x=feature_names))
fig.update_layout(
    title="Feature importance (averaged over time)",
    xaxis_title='Feature Names',
    yaxis_title='Mean |SHAP value|',
    template='ggplot2'
)
fig.show()

In [23]:
def financial_metric_growth(df, growth_col):
    df_wide = df.sort_index().pivot(index='fiscal_year', columns='fiscal_period', values=growth_col)
    df_wide['FY_growth'] = df_wide['FY'].pct_change()
    df_wide['Q4'] = df_wide['FY'] - (df_wide['Q1'] + df_wide['Q2'] + df_wide['Q3'])
    quarters = ['Q1', 'Q2', 'Q3', 'Q4']
    for i in range(1, len(quarters)):
        curr = quarters[i]
        prev = quarters[i-1]
        df_wide[f'{curr}_growth'] = ((df_wide[curr] - df_wide[prev]) / df_wide[prev].abs())
    growth_cols = ['Q2_growth', 'Q3_growth', 'Q4_growth', 'FY_growth']
    growth_df = (df_wide[growth_cols].reset_index().melt(id_vars='fiscal_year',value_name='growth_val',var_name='period_growth'))
    growth_df['fiscal_period'] = growth_df['period_growth'].str.replace('_growth', '', regex=False)
    df_merged = df.merge(growth_df, on=['fiscal_year', 'fiscal_period'], how='left')
    return df_merged['growth_val']

In [35]:
from datetime import datetime, timedelta
from polygon import RESTClient
from Components.TickerData import get_market_cap, get_close_price
import pandas as pd

financials = []
current_date = datetime.today()
past_date = current_date - timedelta(days=(365 * 5))

client = RESTClient()
ticker = 'AAPL'
for f in client.vx.list_stock_financials(ticker, filing_date_lte=current_date.strftime("%Y-%m-%d"),
                                         filing_date_gte=past_date.strftime("%Y-%m-%d")):
    financials.append(f)
financials = pd.DataFrame(financials)

import json  # only needed if your column contains JSON *strings*

financials["financials"] = financials["financials"].apply(
    lambda v: v if isinstance(v, dict) else json.loads(v)
)
flat = pd.json_normalize(
    financials["financials"].tolist()
)
flat_filtered = (
    flat
    .filter(like="value")
    #.dropna(axis=1, how="all")   # drop cols that are all missing
)

flat_filtered.index = financials.index
financials = financials.drop(columns=["financials"]).join(flat_filtered).sort_index().fillna(0)

market_caps = []
close_prices = []
for val in financials['end_date'].values:
    market_caps.append(get_market_cap(ticker,val))
    close_prices.append(get_close_price(ticker,val))
financials['market_cap'] = market_caps
financials['share_price'] = close_prices

financials

,cik,company_name,end_date,filing_date,fiscal_period,fiscal_year,source_filing_file_url,source_filing_url,start_date,balance_sheet.assets.value,...,income_statement.participating_securities_distributed_and_undistributed_earnings_loss_basic.value,income_statement.undistributed_earnings_loss_allocated_to_participating_securities_basic.value,income_statement.preferred_stock_dividends_and_other_adjustments.value,income_statement.basic_earnings_per_share.value,income_statement.diluted_earnings_per_share.value,income_statement.basic_average_shares.value,income_statement.diluted_average_shares.value,income_statement.common_stock_dividends.value,market_cap,share_price
0,0000320193,Apple Inc.,2024-12-28,2025-01-31,Q1,2025,http://api.polygon.io/v1/reference/sec/filings...,https://api.polygon.io/v1/reference/sec/filing...,2024-09-29,3.440850e+11,...,0,0,0,2.41,2.40,1.508172e+10,1.515086e+10,0,3.863453e+12,252.200
1,0000320193,Apple Inc.,2024-09-28,2024-11-01,FY,2024,http://api.polygon.io/v1/reference/sec/filings...,https://api.polygon.io/v1/reference/sec/filing...,2023-10-01,3.649800e+11,...,0,0,0,6.11,6.08,1.534378e+10,1.540810e+10,0,3.463350e+12,233.000
2,0000320193,Apple Inc.,2024-06-29,2024-08-02,Q3,2024,http://api.polygon.io/v1/reference/sec/filings...,https://api.polygon.io/v1/reference/sec/filing...,2024-03-31,3.316120e+11,...,0,0,0,1.40,1.40,1.528752e+10,1.534818e+10,0,3.229664e+12,216.750
3,0000320193,Apple Inc.,2024-03-30,2024-05-03,Q2,2024,http://api.polygon.io/v1/reference/sec/filings...,https://api.polygon.io/v1/reference/sec/filing...,2023-12-31,3.374110e+11,...,0,0,0,1.53,1.53,1.540586e+10,1.546471e+10,0,2.647974e+12,170.030
4,0000320193,Apple Inc.,2023-12-30,2024-02-02,Q1,2024,http://api.polygon.io/v1/reference/sec/filings...,https://api.polygon.io/v1/reference/sec/filing...,2023-10-01,3.535140e+11,...,0,0,0,2.19,2.18,1.550976e+10,1.557664e+10,0,2.994371e+12,185.640
5,0000320193,Apple Inc.,2023-09-30,2023-11-03,FY,2023,http://api.polygon.io/v1/reference/sec/filings...,https://api.polygon.io/v1/reference/sec/filing...,2022-09-25,3.525830e+11,...,0,0,0,6.16,6.13,1.574423e+10,1.581255e+10,0,2.676737e+12,173.750
6,0000320193,Apple Inc.,2023-07-01,2023-08-04,Q3,2023,http://api.polygon.io/v1/reference/sec/filings...,https://api.polygon.io/v1/reference/sec/filing...,2023-04-02,3.350380e+11,...,0,0,0,1.27,1.26,1.569761e+10,1.577502e+10,0,3.050896e+12,192.460
7,0000320193,Apple Inc.,2023-04-01,2023-05-05,Q2,2023,http://api.polygon.io/v1/reference/sec/filings...,https://api.polygon.io/v1/reference/sec/filing...,2023-01-01,3.321600e+11,...,0,0,0,1.53,1.52,1.578715e+10,1.584705e+10,0,2.609039e+12,166.170
8,0000320193,Apple Inc.,2022-12-31,2023-02-03,Q1,2023,http://api.polygon.io/v1/reference/sec/filings...,https://api.polygon.io/v1/reference/sec/filing...,2022-09-25,3.467470e+11,...,0,0,0,1.89,1.88,1.589272e+10,1.595572e+10,0,2.066942e+12,125.070
9,0000320193,Apple Inc.,2022-09-24,2022-10-28,FY,2022,http://api.polygon.io/v1/reference/sec/filings...,https://api.polygon.io/v1/reference/sec/filing...,2021-09-26,3.527550e+11,...,0,0,0,6.15,6.11,1.621596e+10,1.632582e+10,0,2.417523e+12,150.770


In [ ]:
# Metrics needed
fundementals = pd.DataFrame()
fundementals['end_date'] = financials['end_date']
fundementals['fiscal_period'] = financials['fiscal_period']
fundementals['fiscal_year'] = financials['fiscal_year']
fundementals['return_on_equity'] = financials['income_statement.net_income_loss.value'] / financials['balance_sheet.equity.value']
fundementals['operating_margin'] = financials['income_statement.operating_income_loss.value'] / financials['income_statement.revenues.value']
fundementals['debt_to_equity'] = financials['balance_sheet.liabilities.value'] / financials['balance_sheet.equity.value']
fundementals['enterprise_value'] = financials['market_cap'] + financials['balance_sheet.liabilities.value'] - financials['balance_sheet.cash.value']
fundementals['ebit'] = financials['income_statement.net_income_loss.value'] + financials['income_statement.income_tax_expense_benefit.value'] + financials['income_statement.interest_and_debt_expense.value']
fundementals['ebitda'] = fundementals['ebit'] + financials['income_statement.depreciation_and_amortization.value']
fundementals['enterprise_value_to_ebitda_ratio'] = fundementals['enterprise_value'] / fundementals['ebitda']
fundementals['pe_ratio'] = financials['share_price'] / financials['income_statement.basic_earnings_per_share.value']
fundementals['free_cash_flow'] = financials['cash_flow_statement.net_cash_flow_from_operating_activities.value'] + financials['income_statement.interest_and_debt_expense.value'] - financials['income_statement.income_tax_expense_benefit.value'] - financials['cash_flow_statement.net_cash_flow_from_investing_activities.value']
fundementals['fcf_per_share'] = fundementals['free_cash_flow'] / financials['income_statement.basic_average_shares.value']
fundementals['price_to_fcf'] = financials['market_cap'] / fundementals['free_cash_flow']
fundementals['peg_ratio'] = fundementals['pe_ratio'] / fundementals['eps_growth']
fundementals['net_margin'] = financials['income_statement.net_income_loss.value'] / financials['income_statement.revenues.value']
financials['book_value_per_share'] = financials['balance_sheet.equity.value'] / financials['income_statement.basic_average_shares.value']
fundementals['book_value_per_share'] = financials['book_value_per_share']
fundementals['price_to_book_ratio'] = financials['share_price'] / financials['book_value_per_share']
fundementals['current_ratio'] = financials['balance_sheet.current_assets.value'] / financials['balance_sheet.liabilities.value']
fundementals['price_to_sales'] = financials['share_price'] / (financials['income_statement.revenues.value'] / financials['income_statement.basic_average_shares.value'])
fundementals['return_on_invested_capital'] = (financials['income_statement.net_income_loss.value'] - financials['income_statement.common_stock_dividends.value']) / (financials['balance_sheet.equity.value'] + financials['balance_sheet.liabilities.value'])
fundementals['gross_margin'] = (financials['income_statement.revenues.value'] - financials['income_statement.costs_and_expenses.value']) / financials['income_statement.revenues.value']
fundementals['working_capital'] = financials['balance_sheet.current_assets.value'] - financials['balance_sheet.current_liabilities.value']
# Line items needed
fundementals['earnings_per_share'] = financials['income_statement.basic_earnings_per_share.value']
fundementals['revenue'] = financials['income_statement.revenues.value']
fundementals['net_income'] = financials['income_statement.net_income_loss.value']
fundementals['total_assets'] = financials['balance_sheet.assets.value']
fundementals['total_liabilities'] = financials['balance_sheet.liabilities.value']
fundementals['total_debt'] = financials['balance_sheet.liabilities.value']
fundementals['current_assets'] = financials['balance_sheet.current_assets.value']
fundementals['total_assets'] = financials['balance_sheet.current_liabilities.value']
fundementals['dividends_and_other_cash_distributions'] = financials['income_statement.preferred_stock_dividends_and_other_adjustments.value']
fundementals['issuance_or_purchase_of_equity_shares'] = financials['cash_flow_statement.net_cash_flow_from_financing_activities.value']
fundementals['outstanding_shares'] = financials['income_statement.basic_average_shares.value']
fundementals['capital_expenditure'] = financials['cash_flow_statement.net_cash_flow_from_investing_activities.value']
fundementals['operating_expense'] = financials['income_statement.operating_expenses.value']
fundementals['cash_and_equivalents'] = financials['balance_sheet.cash.value']
fundementals['shareholders_equity'] = financials['balance_sheet.equity.value']
fundementals['research_and_development'] = financials['income_statement.research_and_development.value']
fundementals['goodwill_and_intangible_assets'] = financials['balance_sheet.intangible_assets.value']
fundementals['operating_income'] = financials['income_statement.operating_income_loss.value']
fundementals['depreciation_and_amortization'] = financials['income_statement.depreciation_and_amortization.value']
fundementals['earnings_per_share'] = financials['income_statement.basic_earnings_per_share.value']
fundementals['share_price'] = financials['share_price']
fundementals['intangible_assets'] = financials['balance_sheet.intangible_assets.value']
fundementals

,return_on_equity,operating_margin,debt_to_equity,enterprise_value,ebit,ebitda,evEBITDA,pe_ratio,free_cash_flow,fcf_per_share,price_to_fcf,eps_growth,peg_ratio,net_margin,revenue_growth,book_value_per_share,book_value_growth,price_to_book,current_ratio,price_to_sales
0,0.544204,0.344586,4.154214,4.140780e+12,4.258400e+10,4.258400e+10,97.237934,104.647303,1.388900e+10,0.920916,278.166405,NaN,NaN,0.292277,NaN,4.426417,NaN,56.976105,0.480444,30.600248
1,1.645935,0.315102,5.408780,3.771380e+12,1.234850e+11,1.234850e+11,30.541202,38.134206,8.557000e+10,5.576852,40.473885,-0.811688,-46.981342,0.239713,2.021994,3.711601,-5.969325,62.776145,0.496663,9.142664
2,0.321521,0.295557,3.971098,3.494568e+12,2.549400e+10,2.549400e+10,137.074149,154.821429,2.493900e+10,1.631331,129.502560,-8.496732,-18.221291,0.250044,-5.483014,4.363559,-9.393803,49.672756,0.473511,38.630054
3,0.318570,0.307428,3.547686,2.911191e+12,2.805800e+10,2.805800e+10,103.756175,111.130719,1.857800e+10,1.205905,142.532767,-30.136986,-3.687519,0.260443,-24.103701,4.815961,0.802176,35.305519,0.487871,28.863593
4,0.457706,0.337637,3.770769,3.273785e+12,4.032300e+10,4.032300e+10,81.189032,84.767123,3.156100e+10,2.034912,94.875680,NaN,NaN,0.283638,NaN,4.777636,NaN,38.856038,0.514262,24.078883
5,1.560760,0.298214,4.673462,2.967174e+12,1.137360e+11,1.137360e+11,26.088256,28.206169,9.009700e+10,5.722541,29.709500,0.162602,173.467938,0.253062,-2.800461,3.947224,26.318345,44.018282,0.494310,7.137144
6,0.329844,0.281159,4.558582,3.325660e+12,2.273300e+10,2.273300e+10,146.292189,151.543307,2.309100e+10,1.470988,132.124911,-16.993464,-8.917741,0.243053,-13.748998,3.839692,-2.477869,50.123814,0.446416,36.934885
7,0.388687,0.298600,4.343801,2.879041e+12,2.838200e+10,2.838200e+10,101.438972,108.607843,2.201900e+10,1.394742,118.490344,-19.047619,-5.701912,0.254756,-19.050139,3.937252,10.306647,42.204565,0.418193,27.661978
8,0.528813,0.307424,5.112557,2.356962e+12,3.562300e+10,3.562300e+10,66.164045,66.174603,2.982500e+10,1.876645,69.302323,NaN,NaN,0.256056,NaN,3.569369,NaN,35.039802,0.444028,16.966581
9,1.969589,0.302887,5.961537,2.719606e+12,1.191030e+11,1.191030e+11,22.834070,24.515447,1.252050e+11,7.721096,19.308520,8.465608,2.895887,0.253096,7.793788,3.124822,-17.279277,48.249146,0.448238,6.200120


In [ ]:
import os
# %%
# Import stock_data dataframe into an Azure SQL database table using SQLAlchemy
#upload_data_sql(stock_data,"SNP600_1day")
#SNP500_1day = fetch_sql_data('SNP500_1day')
#SNP600_1day = fetch_sql_data('SNP600_1day')
#russell2000_1day = fetch_sql_data('russell2000_1day')
#dowjones_1day = fetch_sql_data('dowjones_1day')
#nasdaq_1day = fetch_sql_data('nasdaq_1day')
#stock_data = pd.concat([SNP500_1day, SNP600_1day, dowjones_1day, nasdaq_1day], ignore_index=True)
# Remove duplicates based on the 'Date' and 'Ticker' columns
#stock_data = stock_data[~stock_data.index.duplicated(keep='first')]
# Before conversion
#print("Column types before:", [type(col).__name__ for col in training_data.columns])

# Apply conversion
#training_data.columns = [str(col) for col in training_data.columns]

# After conversion
#print("Column types after:", [type(col).__name__ for col in training_data.columns])